In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# Indicator Creation

In this notebook we will create new interesting features which we believe to be relevant for clustering purposes. The indicators we will try to create in this notebook are:
1. Account age in days (From account creation till the release of this dataset) (datetime(2022,9,29,11,0,0)) √
2. Account tweets number √
3. Account highest daily tweet count √
4. Account average tweet count √
5. Account tweet number over periods of time (e.g. Covid-19) √ (Partially, we can make other days like the day elon declared he wants to buy twitter)
6. Account tweet hashtag average √
7. Account average tweet text length √
8. Account average number of mentions per tweet √
9. Special Characters √
    1. Tweet number of special characters √
    2. Account average number of special characters in tweets √
10. Account's Entropy
11. Account Number of likes √
12. Account Average of likes √
13. Account Number of comments √
14. Account Average of comments √
15. Account discussion creation score (Tweet to retweet ratio) √
16. Ratio between number of tweets and number of likes √
17. Ratio between number of tweets and number of comments √

In [21]:
users_df = pd.read_csv("dataset/users_dataset_cleaned.csv")
merged_df = pd.read_csv("dataset/merged_dataset.csv")

/var/folders/83/4hrtk21x7lq8yn57wngsh6zm0000gn/T/ipykernel_62019/1226208977.py:2: DtypeWarning: Columns (0,1,2,4,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("dataset/merged_dataset.csv")


## 1. Account age in days
Creating the account age in days feature and adding it to the dataframe.

In [28]:
users_df["created_at"] = pd.to_datetime(users_df["created_at"], errors="coerce") # Even though this is already done in the previous notebook, this information is turned to string when converting the merged_df to csv
release_of_dataset = datetime(2022,9,29,11,0,0)
users_df["account_age_in_days"] = (release_of_dataset - users_df['created_at']).dt.days

users_df.head()

# Keeping info in merged_df too for further calculations
merged_df["account_created"] = pd.to_datetime(merged_df["account_created"], errors="coerce")
merged_df["account_age_in_days"] = (release_of_dataset - merged_df['account_created']).dt.days

# 2. Account number of tweets

Creating the account average tweets per day and adding it to the dataframe.

In [23]:
users_df = users_df.merge(merged_df.groupby('user_id').size().reset_index(name='counts'), how="left")
users_df.rename(columns={"counts": "number_of_tweets"}, inplace=True)
users_df.head()

,user_id,name,lang,bot,created_at,statuses_count,account_age_in_days,number_of_tweets
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,1314,NaN
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,1311,NaN
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,2709,NaN
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,2080,NaN
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,1198,NaN


# 2. Account average tweets per day

Creating the account average tweets per day and adding it to the dataframe.

In [33]:
# list_of_average_tweets_per_day = []

# for elem in merged_df["statuses_count"]:
#     try:
#         tweets_made = elem
#         age_in_days = merged_df["account_age_in_days"][elem]
#         average_tweets_per_day = int(tweets_made)/int(age_in_days)
#         list_of_average_tweets_per_day.append(average_tweets_per_day)
#     except KeyError:
#         list_of_average_tweets_per_day.append(np.nan)

merged_df["account_average_tweets_per_day"] = merged_df["statuses_count"]/merged_df["account_age_in_days"]

# merged_df["account_average_tweets_per_day"] = list_of_average_tweets_per_day
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455680 entries, 0 to 3455679
Data columns (total 17 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   user_id                         object        
 1   name                            object        
 2   lang                            object        
 3   bot                             float64       
 4   account_created                 datetime64[ns]
 5   statuses_count                  float64       
 6   tweet_id                        float64       
 7   retweet_count                   float64       
 8   reply_count                     float64       
 9   favorite_count                  float64       
 10  num_hashtags                    float64       
 11  num_urls                        float64       
 12  num_mentions                    float64       
 13  tweet_created                   object        
 14  text                            object        
 15

,user_id,name,lang,bot,account_created,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,tweet_created,text,account_age_in_days,account_average_tweets_per_day


Idea, isn't better to add this attribute to the users_df ? Because it's a user caractherization, and having it in the merged_df is gonna be redundant

In [34]:
# list_of_average_tweets_per_day = []

# for elem in users_df["statuses_count"]:
#     try:
#         tweets_made = elem
#         age_in_days = users_df["account_age_in_days"][elem]
#         average_tweets_per_day = int(tweets_made)/int(age_in_days)
#         list_of_average_tweets_per_day.append(average_tweets_per_day)
#     except KeyError:
#         list_of_average_tweets_per_day.append(np.nan)

users_df["account_average_tweets_per_day"] = users_df["statuses_count"]/users_df["account_age_in_days"]

# users_df["account_average_tweets_per_day"] = list_of_average_tweets_per_day
users_df.head()

,user_id,name,lang,bot,created_at,statuses_count,account_age_in_days,number_of_tweets,account_average_tweets_per_day
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,1314,NaN,0.057839
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,1311,NaN,0.041190
2,137959629,Dadan Syarifudin,en,1,2015-04-30 07:09:56,3,2709,NaN,0.001107
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,2080,NaN,0.024038
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,1198,NaN,5.914023


We need to know the tweet age in days, the average number of tweets between the days that the user used twitter

In [38]:
# Creating the tweet age in days (for each) till the release of dataset
merged_df['tweet_created'] = merged_df['tweet_created'].apply(pd.to_datetime)
release_of_dataset = datetime(2022,9,29,11,0,0)

merged_df['tweet_created_just_days'] = merged_df['tweet_created'].dt.date
merged_df["tweet_created_just_days"] = pd.to_datetime(merged_df["tweet_created_just_days"], errors="coerce")
merged_df['tweet_age_in_days'] = (release_of_dataset - merged_df['tweet_created_just_days']).dt.days

merged_df.drop(columns="tweet_created_just_days", inplace=True)
merged_df.head()

,user_id,name,lang,bot,account_created,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,tweet_created,text,account_age_in_days,account_average_tweets_per_day,tweet_age_in_days
0,2353593986,Lamonica Raborn,en,1.0,2019-02-22 18:00:42,76.0,4.867187e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu...",1314.0,0.057839,1176.0
1,2353593986,Lamonica Raborn,en,1.0,2019-02-22 18:00:42,76.0,4.795354e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-06-21 08:05:13,"""Happiness is not a station you arrive at, but...",1314.0,0.057839,1196.0
2,2353593986,Lamonica Raborn,en,1.0,2019-02-22 18:00:42,76.0,4.517009e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-05 12:41:00,Music flow.,1314.0,0.057839,1273.0
3,2353593986,Lamonica Raborn,en,1.0,2019-02-22 18:00:42,76.0,4.919200e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-25 12:17:31,@_SimplyKC follow meeee...,1314.0,0.057839,1162.0
4,2353593986,Lamonica Raborn,en,1.0,2019-02-22 18:00:42,76.0,4.605225e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-29 20:55:05,"""You are the only person on earth who can use ...",1314.0,0.057839,1249.0


Adding indicator for average number of tweets per day in which the account ACTUALLY tweeted - Gianluca 

In [ ]:
tmp = merged_df.groupby(["user_id", "tweet_age_in_days"], as_index=False).size()

# computing count
tmp = pd.DataFrame(tmp[["user_id", "tweet_age_in_days", "name"]])
count_df = tmp.rename(columns={"name": "count"})

# computing average
tmp = count_df.groupby(["user_id"], as_index=False).mean()
tmp=pd.DataFrame(tmp[["user_id", "count"]])
avg_df = tmp.rename(columns={"count": "avg", "user_id": "id"})

In [ ]:
# adding the computed avg to users dataframe
#users_df = users_df.merge(avg_df, on="id")
users_df.head()

## 3. Account highest daily tweet count

Creating the account highest daily tweet count and adding it too the dataframe.

In [ ]:
# Trying to see how the group by num of days works
merged_df.groupby(["user_id", "tweet_age_in_days"], as_index=False).max()

In [ ]:
users_df

## 5. Tweet num over period of time

Creating the account tweet num by period of time

In [193]:
covid19_start = release_of_dataset - datetime(2020, 3, 11)
covid19_ipotetic_end_of_restrictions = release_of_dataset - datetime(2021, 12, 15)
covid19_start = covid19_start.days
covid19_ipotetic_end_of_restrictions = covid19_ipotetic_end_of_restrictions.days
temp = merged_df[merged_df["tweet_age_in_days"]<=covid19_start]
temp = temp[temp["tweet_age_in_days"]>covid19_ipotetic_end_of_restrictions]
temp = temp.groupby("user_id", as_index=False).size()
temp = temp.rename(columns={"size": "covid19_num_of_tweets"})
users_df = users_df.merge(temp, on="user_id", how='left')
users_df["covid19_num_of_tweets"].replace(np.nan, 0, inplace=True)
users_df


,user_id,name,lang,bot,created_at,statuses_count,account_age_in_days,number_of_tweets,account_average_tweets_per_day,account_avg_tweets_per_actual_days,...,tweet_num_likes_ratio,tweet_num_replies_ratio,total_retweet_count_x,account_discussion_creation_ratio,total_retweet_count_y,total_retweet_count,avg_special_char_in_text,covid19_num_of_tweets_x,covid19_num_of_tweets_y,covid19_num_of_tweets
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,1314,126,0.057839,4.666667,...,25.200000,0.0,5.0,25.200000,5.0,5.0,14.563492,NaN,NaN,0.0
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,1311,116,0.041190,4.640000,...,19.333333,0.0,3.0,38.666667,3.0,3.0,15.336207,NaN,NaN,0.0
2,0,0,0,0,0,0,0,0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,2080,1358,0.024038,8.035503,...,5.388889,0.0,1187.0,1.144061,1187.0,1187.0,19.241532,NaN,NaN,0.0
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,1198,3434,5.914023,28.380165,...,16.122066,0.0,60.0,57.233333,60.0,60.0,15.024461,1453.0,1453.0,1453.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11104,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1034,1051,1.088975,11.677778,...,17.516667,0.0,6350333.0,0.000166,6350333.0,6350333.0,13.198858,316.0,316.0,316.0
11105,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1616,1975,1.871287,7.397004,...,4.571759,0.0,775759.0,0.002546,775759.0,775759.0,13.324051,215.0,215.0,215.0
11106,0,0,0,0,0,0,0,0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
11107,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,1296,99,0.032407,4.304348,...,11.000000,0.0,7.0,14.142857,7.0,7.0,14.343434,NaN,NaN,0.0


COVID 19 from 11/03/2020 to 15/12/2021 (pretty much the period of time in which we had restictions and had to stay home)

## 6. Average of hashtags per user

Creating the account average of hashtags in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "num_hashtags"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"num_hashtags": "avg_hashtags"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 7. Average tweet text length per user

Creating the account average tweet length

In [ ]:
merged_df["text_length"] = merged_df["text"].str.len() 
tmp = merged_df[["user_id", "text_length"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"text_length": "avg_text_length"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 8. Average of mentions per user

Creating the account average of hashtags in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "num_mentions"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"num_mentions": "avg_mentions"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 9. Average num of special characters in text per user and num of special characters per tweet

Creating the account average of special characters in tweets

In [172]:
import re
merged_df["special_char_in_text"] = merged_df["text"].str.len() - merged_df["text"].str.findall('[\w]').str.len()
tmp = merged_df[["user_id", "special_char_in_text"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"special_char_in_text": "avg_special_char_in_text"})
users_df = users_df.merge(tmp, on="user_id", how='left')
merged_df
users_df

,user_id,name,lang,bot,created_at,statuses_count,account_age_in_days,number_of_tweets,account_average_tweets_per_day,account_avg_tweets_per_actual_days,...,avt_favorite_count,total_replies,avt_reply_count,tweet_num_likes_ratio,tweet_num_replies_ratio,total_retweet_count_x,account_discussion_creation_ratio,total_retweet_count_y,total_retweet_count,avg_special_char_in_text
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76,1314,126,0.057839,4.666667,...,0.040984,0.0,0.0,25.200000,0.0,5.0,25.200000,5.0,5.0,14.563492
1,2358850842,Lourie Botton,en,0,2019-02-26 03:02:32,54,1311,116,0.041190,4.640000,...,0.054545,0.0,0.0,19.333333,0.0,3.0,38.666667,3.0,3.0,15.336207
2,0,0,0,0,0,0,0,0,0.000000,0.000000,...,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,466124818,Carletto Focia,it,1,2017-01-18 02:49:18,50,2080,1358,0.024038,8.035503,...,0.195046,0.0,0.0,5.388889,0.0,1187.0,1.144061,1187.0,1187.0,19.241532
4,2571493866,MBK Ebook,en,0,2019-06-18 19:30:21,7085,1198,3434,5.914023,28.380165,...,0.064742,0.0,0.0,16.122066,0.0,60.0,57.233333,60.0,60.0,15.024461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11104,2911861962,Madrid Lae Maika .,en,0,2019-11-29 13:16:02,1126,1034,1051,1.088975,11.677778,...,0.059880,0.0,0.0,17.516667,0.0,6350333.0,0.000166,6350333.0,6350333.0,13.198858
11105,1378532629,Clau Sato,en,0,2018-04-27 03:01:58,3024,1616,1975,1.871287,7.397004,...,0.227728,0.0,0.0,4.571759,0.0,775759.0,0.002546,775759.0,775759.0,13.324051
11106,0,0,0,0,0,0,0,0,0.000000,0.000000,...,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN
11107,2383025796,Minnie Guadagno,en,1,2019-03-13 02:44:13,42,1296,99,0.032407,4.304348,...,0.095745,0.0,0.0,11.000000,0.0,7.0,14.142857,7.0,7.0,14.343434


## 11. Number of likes per user

Creating the account number of likes in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "favorite_count"]].groupby(["user_id"], as_index=False).sum()
tmp = tmp.rename(columns={"favorite_count": "total_likes"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 12. Average of likes per user

Creating the account average of likes in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "favorite_count"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"favorite_count": "avt_favorite_count"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 13. Number of comments per user

Creating the account number of comments in tweets

In [ ]:
# Trying to see how the group by num of days works
tmp = merged_df[["user_id", "reply_count"]].groupby(["user_id"], as_index=False).sum()
tmp = tmp.rename(columns={"reply_count": "total_replies"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 14. Average of comments per user

Creating the account average of comments in tweets

In [ ]:
tmp = merged_df[["user_id", "reply_count"]].groupby(["user_id"], as_index=False).mean()
tmp = tmp.rename(columns={"reply_count": "avt_reply_count"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df

## 15. Account discussion creation score
Creating the account tweet to retweet ratio

In [ ]:
tmp = merged_df[["user_id", "retweet_count"]].groupby(["user_id"], as_index=False).sum()
tmp = tmp.rename(columns={"retweet_count": "total_retweet_count"})
users_df = users_df.merge(tmp, on="user_id", how='left')
users_df["total_retweet_count"].replace(np.nan, 0, inplace=True)
users_df["account_discussion_creation_ratio"] = users_df["number_of_tweets"]/users_df["total_retweet_count"]
users_df['account_discussion_creation_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
users_df

## 16. Ratio between num of tweets and num of likes

Creating the account ratio between num of tweets and num of likes

In [ ]:
users_df["tweet_num_likes_ratio"] = users_df["number_of_tweets"]/users_df["total_likes"]
users_df['tweet_num_likes_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
users_df

## 17. Ratio between num of tweets and num of comments

Creating the account ratio between num of tweets and num of comments

In [ ]:
users_df["tweet_num_replies_ratio"] = users_df["number_of_tweets"]/users_df["total_replies"] 
users_df['tweet_num_replies_ratio'].replace([np.inf, -np.inf], 0, inplace=True)
users_df